In [ ]:
!git clone https://github.com/anonymousindividual007/Multi-environment-Topic-Models

In [ ]:
!pip install torch pyro-ppl

In [ ]:
import pandas as pd
import numpy as np
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
import nltk
nltk.download('punkt')
from collections import Counter
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from scipy.sparse import csr_matrix

import matplotlib.pyplot as plt
import seaborn as sns
import os
import pyro
import pyro.distributions as dist
from pyro.infer import MCMC, NUTS
import torch
import math
import torch.nn as nn
import torch.nn.functional as F
from pyro.infer import SVI, TraceMeanField_ELBO
from tqdm import trange

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [ ]:
smoke_test = 'CI' in os.environ


In [ ]:
file_path = "/content/Env_Adjusted_TM/data/political_stopwords.txt"

with open(file_path, 'r') as file:
    stopwords_list = file.readlines()


In [ ]:
all_stopwords = [word.strip() for word in stopwords_list]

In [ ]:
len(all_stopwords)

In [ ]:
class LemmaTokenizer:
	def __init__(self):
		self.wnl = WordNetLemmatizer()
	def __call__(self, doc):
		return [t for t in word_tokenize(doc) if str.isalpha(t)]

In [ ]:

file_path = '/content/Multi-environment-Topic-Models/local_channels.csv'

train_data = pd.read_csv(file_path)

test1 = train_data[train_data['source'] == 'right'].sample(frac=0.2, random_state=42)
test2 = train_data[train_data['source'] == 'left'].sample(frac=0.2, random_state=42)

# Drop the sampled rows from train_data
train_data = train_data.drop(test1.index)
train_data = train_data.drop(test2.index)

In [ ]:

# train_data= pd.read_csv('/content/Multi-environment-Topic-Models/channels_ideology_train.csv')
# channels_ideology_test = pd.read_csv('/content/Multi-environment-Topic-Models/channels_ideology_test.csv')
# test1 = channels_ideology_test[channels_ideology_test['source'] == 'Republican']
# test2 = channels_ideology_test[channels_ideology_test['source'] == 'Democratic']
# test3 = channels_ideology_test[channels_ideology_test['source'] == 'balanced']

In [ ]:

# Specify the path to the zip file and the name of the CSV file inside it
# zip_file_path = '/content/Multi-environment-Topic-Models/style_train_large.csv.zip'
# csv_file_name = 'style_train_large.csv'  # Change this if the CSV file has a different name inside the zip

# # Specify the temporary directory to extract the CSV file
# temp_dir = '/content/temp_dir'

# # Create a temporary directory if it doesn't exist
# if not os.path.exists(temp_dir):
#     os.makedirs(temp_dir)

# # Extract the CSV file
# with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
#     zip_ref.extract(csv_file_name, temp_dir)

# # Full path to the extracted CSV file
# csv_file_path = os.path.join(temp_dir, csv_file_name)

# # Load the CSV file into a Pandas DataFrame
# train_data = pd.read_csv(csv_file_path, encoding='ISO-8859-1')
# style_test_df = pd.read_csv('/content/Multi-environment-Topic-Models/style_test.csv', encoding='ISO-8859-1')

In [ ]:


vectorizer = CountVectorizer(tokenizer=LemmaTokenizer(), ngram_range=(1, 1), stop_words=all_stopwords, max_df=0.4, min_df=0.0006)

docs_word_matrix_raw = vectorizer.fit_transform(train_data['text'])
docs_word_matrix_tensor = torch.from_numpy(docs_word_matrix_raw.toarray()).float().to(device)

In [ ]:

#style tok iid
# vectorizer = CountVectorizer(tokenizer=LemmaTokenizer(), ngram_range=(1, 1), stop_words=all_stopwords, max_df=0.5, min_df=0.006)


# docs_word_matrix_raw = vectorizer.fit_transform(train_data['text'])
# docs_word_matrix_tensor = torch.from_numpy(docs_word_matrix_raw.toarray()).float().to(device)

In [ ]:

# vectorizer = CountVectorizer(tokenizer=LemmaTokenizer(),
#                              ngram_range=(1, 1),
#                              stop_words=all_stopwords,
#                              max_df=0.5,
#                              min_df=0.006)

# vectorizer.fit(train_data['text'])

# docs_word_matrix_raw = vectorizer.transform(train_data['text'])

# env_mapping = {value: index for index, value in enumerate(train_data['source'].unique())}
# env_index = train_data['source'].apply(lambda x: env_mapping[x])

# docs_word_matrix_tensor = torch.from_numpy(docs_word_matrix_raw.toarray()).float().to(device)
# env_index_tensor = torch.from_numpy(env_index.to_numpy()).long().to(device)


In [ ]:
vocab = pd.DataFrame(columns=['word', 'index'])
vocab['word'] = vectorizer.get_feature_names_out()
vocab['index'] = vocab.index

In [ ]:
print('Dictionary size: %d' % len(vocab))
print('Corpus size: {}'.format(docs.shape))

In [ ]:
class Encoder(nn.Module):
    # Base class for the encoder net, used in the guide
    def __init__(self, vocab_size, num_topics, hidden, dropout):
        super().__init__()
        self.drop = nn.Dropout(dropout)  # to avoid component collapse
        self.fc1 = nn.Linear(vocab_size, hidden)
        self.fc2 = nn.Linear(hidden, hidden)
        self.fcmu = nn.Linear(hidden, num_topics)
        self.fclv = nn.Linear(hidden, num_topics)
        # NB: here we set `affine=False` to reduce the number of learning parameters
        # See https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm1d.html
        # for the effect of this flag in BatchNorm1d
        self.bnmu = nn.BatchNorm1d(num_topics, affine=False)  # to avoid component collapse
        self.bnlv = nn.BatchNorm1d(num_topics, affine=False)  # to avoid component collapse

    def forward(self, inputs):
        h = F.softplus(self.fc1(inputs))
        h = F.softplus(self.fc2(h))
        h = self.drop(h)
        # μ and Σ are the outputs
        logtheta_loc = self.bnmu(self.fcmu(h))
        logtheta_logvar = self.bnlv(self.fclv(h))
        logtheta_scale = (0.5 * logtheta_logvar).exp()  # Enforces positivity
        return logtheta_loc, logtheta_scale


class Decoder(nn.Module):
    # Base class for the decoder net, used in the model
    def __init__(self, vocab_size, num_topics, dropout):
        super().__init__()
        self.beta = nn.Linear(num_topics, vocab_size, bias=False)
        self.bn = nn.BatchNorm1d(vocab_size, affine=False)
        self.drop = nn.Dropout(dropout)

    def forward(self, inputs):
        inputs = self.drop(inputs)
        # the output is σ(βθ)
        return F.softmax(self.bn(self.beta(inputs)), dim=1)


class ProdLDA(nn.Module):
    def __init__(self, vocab_size, num_topics, hidden, dropout):
        super().__init__()
        self.vocab_size = vocab_size
        self.num_topics = num_topics
        self.encoder = Encoder(vocab_size, num_topics, hidden, dropout)
        self.decoder = Decoder(vocab_size, num_topics, dropout)

    def model(self, docs):
        pyro.module("decoder", self.decoder)
        with pyro.plate("documents", docs.shape[0]):
            # Dirichlet prior 𝑝(𝜃|𝛼) is replaced by a logistic-normal distribution
            logtheta_loc = docs.new_zeros((docs.shape[0], self.num_topics))
            logtheta_scale = docs.new_ones((docs.shape[0], self.num_topics))
            logtheta = pyro.sample(
                "logtheta", dist.Normal(logtheta_loc, logtheta_scale).to_event(1))
            theta = F.softmax(logtheta, -1)

            # conditional distribution of 𝑤𝑛 is defined as
            # 𝑤𝑛|𝛽,𝜃 ~ Categorical(𝜎(𝛽𝜃))
            count_param = self.decoder(theta)
            # Currently, PyTorch Multinomial requires `total_count` to be homogeneous.
            # Because the numbers of words across documents can vary,
            # we will use the maximum count accross documents here.
            # This does not affect the result because Multinomial.log_prob does
            # not require `total_count` to evaluate the log probability.
            total_count = int(docs.sum(-1).max())
            pyro.sample(
                'obs',
                dist.Multinomial(total_count, count_param),
                obs=docs
            )

    def guide(self, docs):
        pyro.module("encoder", self.encoder)
        with pyro.plate("documents", docs.shape[0]):
            # Dirichlet prior 𝑝(𝜃|𝛼) is replaced by a logistic-normal distribution,
            # where μ and Σ are the encoder network outputs
            logtheta_loc, logtheta_scale = self.encoder(docs)
            logtheta = pyro.sample(
                "logtheta", dist.Normal(logtheta_loc, logtheta_scale).to_event(1))

    def beta(self):
        # beta matrix elements are the weights of the FC layer on the decoder
        return self.decoder.beta.weight.cpu().detach().T

In [ ]:
# setting global variables
seed = 0
torch.manual_seed(seed)
pyro.set_rng_seed(seed)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

num_topics = 20 if not smoke_test else 3
docs = docs.float().to(device)
batch_size = 32
learning_rate = 1e-3
num_epochs = 50 if not smoke_test else 1

In [ ]:
# training
pyro.clear_param_store()

prodLDA = ProdLDA(
    vocab_size=docs.shape[1],
    num_topics=num_topics,
    hidden=100 if not smoke_test else 10,
    dropout=0.2
)
prodLDA.to(device)

optimizer = pyro.optim.Adam({"lr": learning_rate})
svi = SVI(prodLDA.model, prodLDA.guide, optimizer, loss=TraceMeanField_ELBO())
num_batches = int(math.ceil(docs.shape[0] / batch_size)) if not smoke_test else 1

bar = trange(num_epochs)
for epoch in bar:
    running_loss = 0.0
    for i in range(num_batches):
        batch_docs = docs[i * batch_size:(i + 1) * batch_size, :]
        loss = svi.step(batch_docs)
        running_loss += loss / batch_docs.size(0)

    bar.set_postfix(epoch_loss='{:.2e}'.format(running_loss))

In [ ]:
test_data_word_matrix_raw = vectorizer.transform(test4['text'])
test_data_word_matrix_tensor = torch.from_numpy(test_data_word_matrix_raw.toarray()).float().to(device)


In [ ]:
test_data_word_matrix_tensor.shape

In [ ]:
import torch
from torch.distributions import Normal

def calculate_perplexity(model, docs):
    """
    Calculate the perplexity of the ProdLDA model on the given documents.

    Args:
        model (ProdLDA): The trained ProdLDA model.
        docs (torch.Tensor): Tensor of documents (word counts) with shape [num_docs, vocab_size].

    Returns:
        float: The perplexity of the model on the documents.
    """
    # Put the model in evaluation mode
    model.eval()

    # No need for gradients
    with torch.no_grad():
        # Use the guide (encoder) to get the parameters of the logistic-normal distribution
        logtheta_loc, logtheta_scale = model.encoder(docs)

        # Draw samples from the logistic-normal distribution
        logtheta = Normal(logtheta_loc, logtheta_scale).sample()
        theta = torch.softmax(logtheta, -1)

        # Decode the samples to get the word distribution
        count_param = model.decoder(theta)

        # Compute log likelihood of the entire dataset given the word distribution
        log_likelihood = torch.sum(docs * (count_param + 1e-10).log())

        # Compute the total number of words in the dataset
        total_word_count = torch.sum(docs)

        # Handle the case where the total_word_count is zero
        if total_word_count == 0:
            raise ValueError("The total word count in the documents is zero. Cannot compute perplexity.")

        # Compute the average negative log likelihood
        avg_neg_log_likelihood = -log_likelihood / total_word_count

        # Compute perplexity
        perplexity = torch.exp(avg_neg_log_likelihood).item()

    return perplexity


In [ ]:
perplexity = calculate_perplexity(prodLDA, test_data_word_matrix_tensor)
print(f'Perplexity: {perplexity}')
